<h1> Git lfs guide </h1>
https://docs.github.com/en/repositories/working-with-files/managing-large-files/installing-git-large-file-storage <br>

Command for lsf when installed -> git lfs track ".csv"

In [ ]:
install = False # Set to True to install the required packages
if install:
   !pip install numpy
   !pip install pandas
   !pip install chess
   !pip install fentoboardimage

<h1> File for testing different prompting styles </h1>

<h2> Importing packages and reading data </h2>

In [8]:
# Importing all relevant and necessary modules
import numpy as np 
import pandas as pd 
import os
import chess
import math
import ollama

#Reading in our training data
df = pd.read_csv(r"chessData.csv")

<h2> Handling generating legal move list and returning a subsection of that list </h2>

In [ ]:
def gen_legal_moves_full(board):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    return legal_moves

def gen_legal_moves(board,frac=1):
    if frac == 1:
        return gen_legal_moves_full(board)
    else:
        #Return a fraction of the legal moves
        legal_moves = gen_legal_moves_full(board)
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

<h2> Testing different prompting styles </h2>

![diagrams\prompt_test_diagrams2.png](diagrams\prompt_test_diagrams2.png)

In [3]:
def init_prompt_test(prompt_list,df,n_test,color):
    results = np.zeros(len(prompt_list))
    for i in range(n_test):
        prompts = gen_prompts(prompt_list,df.iloc[i]['fen'],color)
        result = test_prompt(prompts)
        n_correct += result
    results = (n_test,results)
    output_handler(results)

def gen_prompts(prompt_list,fen,color):
    prompts = []
    board = chess.Board(fen)
    for prompt in prompt_list:
        if prompt[0] == 1:
            prompts.append(gen_prompt1(color,fen))
        else:
            prompts.append(gen_prompt2(color,fen,gen_legal_moves(board,prompt[1])))
    return prompts

def gen_prompt1(color,board, n=10):
    return f"You are playing as {color}. The current board is as follows: \n {board} \n you are playing with {n} other models. Vote on the next move in UCI notation "

def gen_prompt2(color,board,moves, n=10):
    return f"You are playing as {color}. The current board is as follows: \n {board} \n you are playing with {n} other models. Here you are given a list of legal moves {moves}. You have to vote on the next move to make from the list of legal moves and provide the move exactly as they are given to you"
    
def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

def prompt_move(prompt,board):
    response = ollama.chat(model='llama2', messages=[
            {'role': 'user','content': prompt,},])
    move = response['message']['content']
    for k in range(len(move)):
        if move[k:k+4] in gen_legal_moves(board):
            move = move[k:k+4]
    return move

def test_prompt(prompts,board):
    result = np.zeros(len(prompts))
    for i in range(len(prompts)):
        prompt = prompts[i]
        move = prompt_move(prompt)
        if move in  gen_legal_moves(board):
            result[i] = 1
    return result

def output_handler(results):
    pass
def gen_histograms(results):
    pass
def gen_histogram(result):
    pass

IndentationError: expected an indented block after function definition on line 34 (1605804769.py, line 36)